In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv(r'/content/RELIANCE.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-02,314.924988,324.975006,313.000000,321.600006,265.931763,19847448.0
1,2009-01-05,324.750000,344.250000,323.774994,341.462006,282.355743,22027876.0
2,2009-01-06,340.000000,347.500000,332.812012,342.725006,283.400116,24118400.0
3,2009-01-07,342.250000,346.500000,293.750000,300.187988,248.226151,34002200.0
4,2009-01-09,302.500000,302.500000,273.412994,288.312012,238.405838,37952520.0


In [5]:
df.shape

(2634, 7)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2634 entries, 0 to 2633
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2634 non-null   object 
 1   Open       2634 non-null   float64
 2   High       2634 non-null   float64
 3   Low        2634 non-null   float64
 4   Close      2634 non-null   float64
 5   Adj Close  2634 non-null   float64
 6   Volume     2634 non-null   float64
dtypes: float64(6), object(1)
memory usage: 144.2+ KB


In [7]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000,2.634000e+03
mean,593.500057,600.787471,586.714278,593.416973,556.435824,8.622828e+06
std,264.054719,266.530348,261.098518,263.770466,278.365257,5.485415e+06
min,0.000000,286.250000,266.250000,269.388000,222.757553,0.000000e+00
25%,432.568741,437.500000,426.962509,432.125000,391.739197,5.316242e+06
50%,497.174988,502.500000,491.550003,496.649994,445.431839,7.126298e+06
75%,556.181259,565.875000,546.812500,554.640991,524.052490,1.009307e+07
max,1407.949951,1417.500000,1402.650024,1408.849976,1401.091064,8.624427e+07


In [8]:
#Setting dat as index
df.index = pd.to_datetime(df['Date'])
#dropping the date column since it is already set as index
df = df.drop(['Date'], axis=1)
df.head()

<bound method NDFrame.head of                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2009-01-02   314.924988   324.975006   313.000000   321.600006   265.931763   
2009-01-05   324.750000   344.250000   323.774994   341.462006   282.355743   
2009-01-06   340.000000   347.500000   332.812012   342.725006   283.400116   
2009-01-07   342.250000   346.500000   293.750000   300.187988   248.226151   
2009-01-09   302.500000   302.500000   273.412994   288.312012   238.405838   
...                 ...          ...          ...          ...          ...   
2019-09-09  1220.650024  1233.000000  1213.150024  1222.199951  1222.199951   
2019-09-11  1222.500000  1240.000000  1222.500000  1234.400024  1234.400024   
2019-09-12  1235.000000  1240.449951  1205.699951  1210.349976  1210.349976   
2019-09-13  1212.000000  1228.500000  1206.900024  1225.599976  1225.599976   
2019-09-16     0.000000  1219.099976  1186.099976  1210.750000  1210.750000   

                Volume  
Date                    
2009-01-02  19847448.0  
2009-01-05  22027876.0  
2009-01-06  24118400.0  
2009-01-07  34002200.0  
2009-01-09  37952520.0  
...                ...  
2019-09-09   5370758.0  
2019-09-11   5544468.0  
2019-09-12   5431139.0  
2019-09-13   5919260.0  
2019-09-16   9390427.0  

[2634 rows x 6 columns]>